In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from naive_bayes import GaussianNB

from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
# 数据读取
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# read data
train = pd.read_csv('data/train.csv').fillna(' ')
test = pd.read_csv('data/test.csv').fillna(' ')
submission = pd.read_csv('data/sample_submission.csv')

# 单独保存comment_text
train_text = train['comment_text'].str.lower()
test_text = test['comment_text'].str.lower()
# 获得y_train
y_train = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

# 连接所有文字用于分词
all_text = pd.concat([train_text, test_text], axis = 0, ignore_index = True)

In [3]:
y_train.head(10)

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,1,1,1,0,1,0
7,0,0,0,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0


In [4]:
# 原始数据可视化分析
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# 数据预处理
# all_text = all_text[:100]
def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(sentence)):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
        elif tag.startswith('R'):
            yield wnl.lemmatize(word, pos='r')
            
        else:
            yield word

def text_cleaned_process(text_raw):    
    text_raw = str(text_raw)
    text_raw = str(text_raw.lower())
    text_raw = re.sub(r'[^a-zA-Z]', ' ', text_raw)
    
    words = text_raw.split()
    
    # 移除长度小于3的词语
    words2 = []
    for i in words:
        if len(i) >= 0:
            words2.append(i)
    # 去停止词
    stops = set(stopwords.words('english'))
    
    result_text = []
    result_text = " ".join([w for w in words2 if not w in stops])
    
    return(" ".join(lemmatize_all(result_text)))

# 去掉数字
all_text.replace({r'[^\x00-\x7F]+':''},regex=True,inplace=True)

num_all_text = all_text.size

# 输出清洗后的数据
all_text_cleaned = []

for i in range(0, num_all_text):
    all_text_cleaned.append(text_cleaned_process(all_text[i]))

# 构建pd形式
all_text_cleaned = pd.Series(all_text_cleaned)

'\nspecial_character_removal=re.compile(r\'[^a-z\\d ]\',re.IGNORECASE)\nreplace_numbers=re.compile(r\'\\d+\',re.IGNORECASE)\n\ndef text_to_words(text):\n    # 去掉一些特殊字符\n    text = text.split()\n    text = \' \'.join(text)\n    \n    # 进一步去掉特殊字母\n    text=special_character_removal.sub(\'\',text)\n    \n    # 用空格代替数字\n    text=replace_numbers.sub(\'\', text)\n    \n    # 去停止词\n    stops = set(stopwords.words("english"))\n    text = text.split()\n    text = [w for w in text if not w in stops]\n\n    # 去时态后缀\n    \'\'\'\n    stemmer = SnowballStemmer(\'english\')\n    stemmed_words = [stemmer.stem(word) for word in text]\n    \'\'\'\n    text = \' \'.join(text)\n     \n    return text\n\n# 输出清洗后数据\nall_text_cleaned = []\nfor text in all_text:\n    all_text_cleaned.append(text_to_words(text))\n\n# 构建pd格式\nall_text_cleaned = pd.Series(all_text_cleaned)\n'

In [22]:
# 原始数据可视化分析
# all_text = all_text[:100]
print("Len of all_text_cleaned:", len(all_text_cleaned))

print("Text[0] before cleaned: \n", all_text[0])
print("Text[0] after cleaned: \n", all_text_cleaned[0])

Len of all_text_cleaned: 312735
Text[0] before cleaned: 
 explanation
why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.89.205.38.27
Text[0] after cleaned: 
 explanation edits make username hardcore metallica fan revert vandalisms closure gas vote new york doll fac please remove template talk page since retire


In [8]:
# 数据分词处理
# 按词训练
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=10000)
# 训练tf-idf模型
word_vectorizer.fit(all_text_cleaned)
# 获取词向量
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

# 按字母训练
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
# 训练tf-idf模型
char_vectorizer.fit(all_text_cleaned)
# 获取词向量
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

# 向量结合
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

Number of Unique Tokens 278517


In [9]:
# 不需要拆分train数据
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.15, random_state = 255)
display(train_features.head(10))
display(test_features.head(10))

Len of x_train:  159571
Len of y_train:  159571


In [11]:
# 保存得分
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})

# 开始训练
for class_name in class_names:
    # 依次读取训练目标数据
    train_target = train[class_name]
    # 导入机器学习模型
    #classifier = LogisticRegression(C=4, dual=True)
    classifier = SVC(keenel = 'rbf', random_state = 255, gamma = 0.1, C = 0.3)
    # 计算模型的ROC-AUC得分
    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    # 保存得分
    scores.append(cv_score)
    # 输出得分
    print('CV score for class {} is {}'.format(class_name, cv_score))
    
    # 训练模型
    classifier.fit(train_features, train_target)
    # 输出test数据预测概率
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

# 计算所有类型的平均得分
print('Total CV score is {}'.format(np.mean(scores)))



In [ ]:
# 输出结果文件 
submission.to_csv('submission_tf-idf_svm.csv', index=False)

In [21]:
# 初步检查输出结果
submission.head(10)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.997295,0.329954,0.956259,0.097659,0.873318,0.646170
1,0000247867823ef7,0.004758,0.000019,0.000165,0.000068,0.000244,0.000055
2,00013b17ad220c46,0.001637,0.000025,0.000056,0.000035,0.000078,0.000032
3,00017563c3f7919a,0.000263,0.000004,0.000004,0.000009,0.000005,0.000002
4,00017695ad8997eb,0.011265,0.000053,0.000364,0.000140,0.000811,0.000130
5,0001ea8717f6de06,0.000978,0.000005,0.000015,0.000027,0.000028,0.000020
6,00024115d4cbde0f,0.004427,0.000007,0.000080,0.000031,0.000182,0.000034
7,000247e83dcc1211,0.583627,0.002034,0.017909,0.003164,0.174073,0.007318
8,00025358d4737918,0.341595,0.000226,0.003430,0.000644,0.082057,0.001456
9,00026d1092fe71cc,0.000991,0.000006,0.000011,0.000018,0.000025,0.000013
